In [1]:
import pandas as pd
import numpy as np
import unicodedata
import re

In [2]:
data = pd.read_excel("Cats_May 1, 2024_09.38.xlsx")
#Drop irrelevant info


data.drop(['Status', 'Duration (in seconds)', 'DistributionChannel','UserLanguage',
           "IPAddress", "StartDate","EndDate", "Progress",
           "RecordedDate", "ResponseId", "RecipientLastName",
          "RecipientFirstName", "RecipientEmail", "ExternalReference",
          "LocationLatitude", "LocationLongitude"], axis=1, inplace = True)


print("Number of subjects: ", data.shape[0])
n_sub_raw = data.shape[0]

# Remove unfinished survey responses (and count them)
finish_index = data[data.Finished == "False"].index
N_unfinished = finish_index.size
data = data.drop(finish_index)

print("Number of subjects that failed attention check: ", n_sub_raw - data.shape[0])

# remove attention check questions
attention_check_columns = ["T1 Cute Rep#1_1", "T1 Cute Rep#1_2", "T1 Cute Rep#1_3",
           "T2 Cute Rep#1_1", "T2 Cute Rep#1_2", "T2 Cute Rep#1_3",
           "T1 Scary Rep#1_1", "T1 Scary Rep#1_2", "T1 Scary Rep#1_3",
           "T2 Scary Rep#1_1", "T2 Scary Rep#1_2", "T2 Scary Rep#1_3",
           "T1 Dirty Rep#1_1", "T1 Dirty Rep#1_2", "T1 Dirty Rep#1_3",
           "T2 Dirty Rep#1_1", "T2 Dirty Rep#1_2", "T2 Dirty Rep#1_3"]

data.reset_index(inplace=True)
data.drop(['index'], axis = 1, inplace = True)
metadata = pd.DataFrame(data.iloc[0,:]).transpose()
data.drop(0, inplace=True)
data


/home/arashsm79/.miniforge3/envs/neuroimage/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Number of subjects:  149
Number of subjects that failed attention check:  1


,Finished,Age,Gender,Studentship,Study level,Field of study,Occupation,Allergy,Cats previously,Cityness_1,...,FL_7_DO,FL_8_DO,FL_9_DO,FL_10_DO,FL_11_DO,FL_12_DO,FL_13_DO,FL_14_DO,FL_15_DO,FL_16_DO
1,True,23,NaN,Yes,Bachelors degree,psychology,NaN,No,Yes,82,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1sttextdirty |2ndtextneutral,NaN
2,True,22,NaN,Yes,Bachelors degree,psychology,NaN,No,Yes,60,...,NaN,NaN,NaN,NaN,2ndtextdirty|1sttextneutral,NaN,NaN,NaN,NaN,NaN
3,True,21,NaN,Yes,Bachelors degree,Psychology,NaN,Maybe,Yes,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,25,NaN,Yes,Pre-University,Philosophy,NaN,Maybe,No,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,True,22,NaN,Yes,Bachelors degree,Psychology,NaN,No,Yes,30,...,NaN,NaN,NaN,NaN,NaN,1sttextneutral|2ndtextscary,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,True,22,Male,Yes,Bachelors degree,Psychology,NaN,Yes,No,50,...,2ndtextcute|1sttextdirty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144,True,20,Female,Yes,Bachelors degree,Letters,NaN,Yes,No,70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2ndtextneutral|EndSona,NaN,NaN
145,True,19,Female,Yes,Pre-University,psychology,NaN,No,No,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2ndtextneutral|1sttextscary
146,True,24,Male,Yes,Bachelors degree,lettres,NaN,No,No,50,...,NaN,NaN,1sttextscary|2ndtextcute,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Determine the conditions for each block

In [3]:
condition_columns = ['FL_5_DO', 'FL_6_DO', 'FL_7_DO', 'FL_8_DO', 'FL_9_DO', 'FL_10_DO', 'FL_11_DO', 'FL_12_DO', 'FL_13_DO', 'FL_14_DO', 'FL_15_DO', 'FL_16_DO']
first_non_null_values = data[condition_columns].apply(lambda row: row.first_valid_index(), axis=1)
def get_condition(row):
    conditions_raw = re.sub(r'\s+', '', unicodedata.normalize("NFKD", row[row.first_valid_index()]))
    conditions = []
    for cond_raw in conditions_raw.split('|'):
        try:
            cond_text = cond_raw[0]
            cond_image = cond_raw.split('text')[1].capitalize()
            if cond_image == 'Neutral':
                cond_image = 'Neut'
            condition = f"T{cond_text} {cond_image}"
        except Exception:
            condition = 'invalid'
        conditions.append(condition)
    return np.array(conditions)

conditions = data[condition_columns].apply(get_condition, axis=1)
conditions = np.vstack(conditions)
data['condition1'] = conditions[:, 0]
data['condition2'] = conditions[:, 1]
metadata['condition1'] = 'condition1'
metadata['condition2'] = 'condition2'

data.drop(columns=condition_columns + ['FL_4_DO'], inplace=True)
metadata.drop(columns=condition_columns + ['FL_4_DO'], inplace=True)

data

,Finished,Age,Gender,Studentship,Study level,Field of study,Occupation,Allergy,Cats previously,Cityness_1,...,T2 Neut Recog_41,T2 Neut Recog_42,T2 Neut Recog_43,T2 Neut Recog_44,T2 Neut Recog_45,T2 Neut Recog_DO,id,Source,condition1,condition2
1,True,23,NaN,Yes,Bachelors degree,psychology,NaN,No,Yes,82,...,No,No,No,No,No,carcasses|rodent populations|misfortune|cheris...,10432,SONA,T1 Dirty,T2 Neut
2,True,22,NaN,Yes,Bachelors degree,psychology,NaN,No,Yes,60,...,NaN,NaN,NaN,NaN,NaN,NaN,10516,SONA,T2 Dirty,T1 Neut
3,True,21,NaN,Yes,Bachelors degree,Psychology,NaN,Maybe,Yes,60,...,NaN,NaN,NaN,NaN,NaN,NaN,11512,SONA,T1 Cute,T2 Scary
4,True,25,NaN,Yes,Pre-University,Philosophy,NaN,Maybe,No,86,...,NaN,NaN,NaN,NaN,NaN,NaN,11644,SONA,T1 Cute,T2 Dirty
5,True,22,NaN,Yes,Bachelors degree,Psychology,NaN,No,Yes,30,...,NaN,NaN,NaN,NaN,NaN,NaN,8890,SONA,T1 Neut,T2 Scary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,True,22,Male,Yes,Bachelors degree,Psychology,NaN,Yes,No,50,...,NaN,NaN,NaN,NaN,NaN,NaN,11032,SONA,T2 Cute,T1 Dirty
144,True,20,Female,Yes,Bachelors degree,Letters,NaN,Yes,No,70,...,No,No,No,No,No,black|knocking off|bring comfort|dander allerg...,11404,SONA,T2 Neut,invalid
145,True,19,Female,Yes,Pre-University,psychology,NaN,No,No,90,...,No,No,No,No,No,social media|psychic abilities|low-maintenance...,10705,SONA,T2 Neut,T1 Scary
146,True,24,Male,Yes,Bachelors degree,lettres,NaN,No,No,50,...,NaN,NaN,NaN,NaN,NaN,NaN,11704,SONA,T1 Scary,T2 Cute


# Recognition Task

In [4]:
recog_map = pd.read_csv('recog-task.csv')
recog_task_df = pd.DataFrame(columns=['condition1_recog_pos', 'condition1_recog_neg', 'condition1_recog_neut', 'condition1_recog_unrel', 'condition2_recog_pos', 'condition2_recog_neg', 'condition2_recog_neut', 'condition2_recog_unrel'])

total_pos = np.sum(recog_map['valence'] == 'pos')/2
total_neg = np.sum(recog_map['valence'] == 'neg')/2
total_neut = np.sum(recog_map['valence'] == 'neut')/2
total_unrel = np.sum(recog_map['valence'] == 'unrel')/2

for index, row in data.iterrows():

    recog_counts = []
    for condition_column_name in ['condition1', 'condition2']:
        pos = 0
        neg = 0
        neut = 0
        unrel = 0
        condition = row[condition_column_name]
        if condition != 'invalid':
            cond_recog_columns = [data.columns.get_loc(col) for col in data.columns if col.startswith(condition + ' Recog')]
            assert len(cond_recog_columns) == 46
            recog_words_raw = metadata.iloc[:,cond_recog_columns]
            for (column_idx, recog_word_raw) in zip(cond_recog_columns, recog_words_raw.iloc[0,:]):
                answer = row.iloc[column_idx]
                recog_word = recog_word_raw[recog_word_raw.index('-')+1:].strip()
                if recog_word == 'Display Order':
                    continue
                valence = recog_map[recog_map['data'] == recog_word]['valence'].iloc[0]

                if valence == 'pos' and answer == 'Yes':
                    pos += 1
                elif valence == 'neg' and answer == 'Yes':
                    neg += 1
                elif valence == 'neut' and answer == 'Yes':
                    neut += 1
                elif valence == 'unrel' and answer == 'Yes':
                    unrel += 1
        recog_counts.extend([pos/total_pos, neg/total_neg, neut/total_neut, unrel/total_unrel])

    recog_task_df.loc[len(recog_task_df)] = recog_counts


recog_task_df

,condition1_recog_pos,condition1_recog_neg,condition1_recog_neut,condition1_recog_unrel,condition2_recog_pos,condition2_recog_neg,condition2_recog_neut,condition2_recog_unrel
0,0.666667,0.583333,1.000000,0.000000,0.500000,0.750000,0.666667,0.000000
1,1.000000,0.916667,0.666667,0.133333,1.000000,0.833333,0.666667,0.200000
2,0.916667,0.583333,0.666667,0.133333,0.583333,0.916667,0.833333,0.066667
3,0.916667,0.583333,0.333333,0.066667,0.750000,0.083333,0.666667,0.133333
4,0.833333,0.833333,0.666667,0.066667,0.416667,0.750000,0.833333,0.000000
...,...,...,...,...,...,...,...,...
142,0.500000,0.333333,0.333333,0.200000,0.500000,0.583333,0.333333,0.266667
143,0.583333,0.916667,0.500000,0.066667,0.000000,0.000000,0.000000,0.000000
144,0.666667,0.833333,0.500000,0.000000,1.000000,0.833333,1.000000,0.066667
145,0.916667,0.666667,0.833333,0.133333,0.833333,0.833333,0.833333,0.000000


# Free Recall

In [5]:
freerecall_task_df = pd.DataFrame(columns=['condition1_freerecall', 'condition1_freerecall_pos', 'condition1_freerecall_neg', 'condition1_freerecall_neut', 'condition1_freerecall_unrel', 'condition2_freerecall', 'condition2_freerecall_pos', 'condition2_freerecall_neg', 'condition2_freerecall_neut', 'condition2_freerecall_unrel'])

for index, row in data.iterrows():
    freerecall_counts = []
    for condition_column_name in ['condition1', 'condition2']:
        pos = 0
        neg = 0
        neut = 0
        unrel = 0
        condition = row[condition_column_name]
        freerecall_text = ''
        if condition != 'invalid':
            cond_freerecall_columns = [data.columns.get_loc(col) for col in data.columns if col.startswith(condition + ' Free Recall')]
            assert len(cond_freerecall_columns) == 1
            cond_freerecall_column_idx = cond_freerecall_columns[0]
            freerecall_text = row.iloc[cond_freerecall_column_idx]

        freerecall_counts.extend([freerecall_text, pos, neg, neut, unrel])

    freerecall_task_df.loc[len(freerecall_task_df)] = freerecall_counts

freerecall_task_df

,condition1_freerecall,condition1_freerecall_pos,condition1_freerecall_neg,condition1_freerecall_neut,condition1_freerecall_unrel,condition2_freerecall,condition2_freerecall_pos,condition2_freerecall_neg,condition2_freerecall_neut,condition2_freerecall_unrel
0,We domesticated cats starting 10'000 years ago...,0,0,0,0,"In the middle ages, cats were seen as incarnat...",0,0,0,0
1,They were seen as a sign of misfortune in the ...,0,0,0,0,The cats were really appreciated by the Egypti...,0,0,0,0
2,Les chats étaient considérés comme des dieux e...,0,0,0,0,"Dans le passé, les chats pouvaient être synony...",0,0,0,0
3,"They can spread diseases, they are dangerous f...",0,0,0,0,"Cats can become known on social media, their s...",0,0,0,0
4,Cats kill about 30 millions birds in Switzerla...,0,0,0,0,Cats are responsibles for the extinction of 63...,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
142,"80 species, allergies are transmitted through ...",0,0,0,0,"cats are seen as gods in forein countries, if ...",0,0,0,0
143,"They can cause allergie to 10-20% humain , the...",0,0,0,0,,0,0,0,0
144,"they boost our serotonin levels, they were a s...",0,0,0,0,"considered like goddesses in ancient Egypt, th...",0,0,0,0
145,they were revered in ancient egypypt \nstudies...,0,0,0,0,their consumption of carcasses and spoiled foo...,0,0,0,0


# Spam Check

In [6]:
# Remove wrong answers to attention check
def attention_answer_is_wrong(condition_name, answer):
    is_wrong = False
    if condition_name == "T1 Cute Rep#1_2" and answer == "No":
        is_wrong = True
    elif condition_name == "T1 Cute Rep#1_3" and answer == "No":
        is_wrong = True
    elif condition_name == "T1 Scary Rep#1_1" and answer == "No":
        is_wrong = True
    elif condition_name == "T1 Scary Rep#1_2" and answer == "Yes":
        is_wrong = True
    elif condition_name == "T1 Dirty Rep#1_1" and answer == "No":
        is_wrong = True
    elif condition_name == "T1 Dirty Rep#1_2" and answer == "Yes":
        is_wrong = True
    elif condition_name == "T2 Cute Rep#1_2" and answer == "No":
        is_wrong = True
    elif condition_name == "T2 Cute Rep#1_3" and answer == "No":
        is_wrong = True
    elif condition_name == "T2 Scary Rep#1_1" and answer == "No":
        is_wrong = True
    elif condition_name == "T2 Scary Rep#1_2" and answer == "Yes":
        is_wrong = True
    elif condition_name == "T2 Dirty Rep#1_1" and answer == "No":
        is_wrong = True
    elif condition_name == "T2 Dirty Rep#1_2" and answer == "Yes":
        is_wrong = True
    return is_wrong

spam_df = pd.DataFrame(columns=['wrong_answers'])

for index, row in data.iterrows():
    wrong_count = 0
    for condition_column_name in ['condition1', 'condition2']:
        condition = row[condition_column_name]
        if condition != 'invalid' and 'Neut' not in condition:
            cond_spam_columns = [col for col in data.columns if col.startswith(condition + ' Rep#')]
            assert len(cond_spam_columns) == 3
            for column_name in cond_spam_columns:
                answer = row[column_name]
                if attention_answer_is_wrong(column_name, answer):
                    wrong_count += 1

    spam_df.loc[len(spam_df)] = [wrong_count]


def column_statistics(data, column_name):
    column_data = data[column_name]
    statistics = {
        'Mean': column_data.mean(),
        'Median': column_data.median(),
        'Mode': column_data.mode()[0],
        'Standard Deviation': column_data.std(),
        'Minimum': column_data.min(),
        'Maximum': column_data.max(),
        'Count': column_data.count(),
        'Unique Values': column_data.nunique()
    }
    return statistics

column_statistics(spam_df[spam_df['wrong_answers'] > 0],  'wrong_answers')

cond1_col = data['condition1']
cond2_col = data['condition2']

covariates = data.iloc[:, 1:12].copy()
study_df = pd.concat([covariates, spam_df, recog_task_df, freerecall_task_df], axis=1)
study_df['condition1'] = cond1_col
study_df['condition2'] = cond2_col
study_df = study_df[study_df['wrong_answers'] < 2]
study_df.reset_index(inplace=True, drop=True)
study_df

,Age,Gender,Studentship,Study level,Field of study,Occupation,Allergy,Cats previously,Cityness_1,Nationality,...,condition1_freerecall_neg,condition1_freerecall_neut,condition1_freerecall_unrel,condition2_freerecall,condition2_freerecall_pos,condition2_freerecall_neg,condition2_freerecall_neut,condition2_freerecall_unrel,condition1,condition2
0,23,NaN,Yes,Bachelors degree,psychology,NaN,No,Yes,82,UK,...,0.0,0.0,0.0,The cats were really appreciated by the Egypti...,0.0,0.0,0.0,0.0,T1 Dirty,T2 Neut
1,22,NaN,Yes,Bachelors degree,psychology,NaN,No,Yes,60,Swiss,...,0.0,0.0,0.0,"Dans le passé, les chats pouvaient être synony...",0.0,0.0,0.0,0.0,T2 Dirty,T1 Neut
2,21,NaN,Yes,Bachelors degree,Psychology,NaN,Maybe,Yes,60,Swiss,...,0.0,0.0,0.0,"Cats can become known on social media, their s...",0.0,0.0,0.0,0.0,T1 Cute,T2 Scary
3,25,NaN,Yes,Pre-University,Philosophy,NaN,Maybe,No,86,Swiss and argentinian,...,0.0,0.0,0.0,Cats are responsibles for the extinction of 63...,0.0,0.0,0.0,0.0,T1 Cute,T2 Dirty
4,22,NaN,Yes,Bachelors degree,Psychology,NaN,No,Yes,30,Swiss,...,0.0,0.0,0.0,Cats were loved by egyptians and they were mum...,0.0,0.0,0.0,0.0,T1 Neut,T2 Scary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,22,Male,Yes,Bachelors degree,Psychology,NaN,Yes,No,50,Swiss,...,0.0,0.0,0.0,,0.0,0.0,0.0,0.0,T2 Cute,T1 Dirty
139,20,Female,Yes,Bachelors degree,Letters,NaN,Yes,No,70,Portuguese,...,0.0,0.0,0.0,"considered like goddesses in ancient Egypt, th...",0.0,0.0,0.0,0.0,T2 Neut,invalid
140,19,Female,Yes,Pre-University,psychology,NaN,No,No,90,Portuguese,...,0.0,0.0,0.0,their consumption of carcasses and spoiled foo...,0.0,0.0,0.0,0.0,T2 Neut,T1 Scary
141,24,Male,Yes,Bachelors degree,lettres,NaN,No,No,50,suisse,...,0.0,0.0,0.0,cats were mummified in ancient egypt\ncats hav...,0.0,0.0,0.0,0.0,T1 Scary,T2 Cute


In [7]:
study_df.to_csv("preprocessed_data.csv")

# Merge Manual Free Recall Counts

In [8]:
t1_1 = pd.read_excel("manual_annotation/oliver_t1.xlsx")[['condition1_freerecall_pos', 'condition1_freerecall_neg', 'condition1_freerecall_neut', 'condition1_freerecall_unrel']]
t1_1.fillna(0, inplace=True)
t1_2 = pd.read_excel("manual_annotation/amine_t1.xlsx")[['condition1_freerecall_pos', 'condition1_freerecall_neg', 'condition1_freerecall_neut', 'condition1_freerecall_unrel']]
t1_2.fillna(0, inplace=True)
t1 = (t1_1 + t1_2) / 2

study_df[['condition1_freerecall_pos', 'condition1_freerecall_neg', 'condition1_freerecall_neut', 'condition1_freerecall_unrel']] = t1

t2_1 = pd.read_excel("manual_annotation/andrey_t2.xlsx")[['condition2_freerecall_pos', 'condition2_freerecall_neg', 'condition2_freerecall_neut', 'condition2_freerecall_unrel']]
t2_1.fillna(0, inplace=True)
t2_2 = pd.read_csv("manual_annotation/valentine_t2.csv")[['condition2_freerecall_pos', 'condition2_freerecall_neg', 'condition2_freerecall_neut', 'condition2_freerecall_unrel']]
t2 = (t2_1 + t2_2) / 2

study_df[['condition2_freerecall_pos', 'condition2_freerecall_neg', 'condition2_freerecall_neut', 'condition2_freerecall_unrel']] = t2

study_df

,Age,Gender,Studentship,Study level,Field of study,Occupation,Allergy,Cats previously,Cityness_1,Nationality,...,condition1_freerecall_neg,condition1_freerecall_neut,condition1_freerecall_unrel,condition2_freerecall,condition2_freerecall_pos,condition2_freerecall_neg,condition2_freerecall_neut,condition2_freerecall_unrel,condition1,condition2
0,23,NaN,Yes,Bachelors degree,psychology,NaN,No,Yes,82,UK,...,1.0,1.0,0.0,The cats were really appreciated by the Egypti...,2.5,1.0,0.0,0.0,T1 Dirty,T2 Neut
1,22,NaN,Yes,Bachelors degree,psychology,NaN,No,Yes,60,Swiss,...,1.0,0.5,0.0,"Dans le passé, les chats pouvaient être synony...",1.0,2.0,0.0,0.0,T2 Dirty,T1 Neut
2,21,NaN,Yes,Bachelors degree,Psychology,NaN,Maybe,Yes,60,Swiss,...,2.0,0.0,0.0,"Cats can become known on social media, their s...",2.0,1.0,0.0,0.0,T1 Cute,T2 Scary
3,25,NaN,Yes,Pre-University,Philosophy,NaN,Maybe,No,86,Swiss and argentinian,...,0.5,0.5,0.0,Cats are responsibles for the extinction of 63...,2.0,1.0,0.0,0.0,T1 Cute,T2 Dirty
4,22,NaN,Yes,Bachelors degree,Psychology,NaN,No,Yes,30,Swiss,...,1.0,0.5,0.0,Cats were loved by egyptians and they were mum...,2.5,1.0,0.0,0.0,T1 Neut,T2 Scary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,22,Male,Yes,Bachelors degree,Psychology,NaN,Yes,No,50,Swiss,...,2.0,0.5,0.5,,0.0,0.0,0.0,0.0,T2 Cute,T1 Dirty
139,20,Female,Yes,Bachelors degree,Letters,NaN,Yes,No,70,Portuguese,...,1.5,0.5,0.0,"considered like goddesses in ancient Egypt, th...",2.0,1.0,0.0,0.0,T2 Neut,invalid
140,19,Female,Yes,Pre-University,psychology,NaN,No,No,90,Portuguese,...,1.0,0.5,0.0,their consumption of carcasses and spoiled foo...,0.0,2.5,0.0,0.0,T2 Neut,T1 Scary
141,24,Male,Yes,Bachelors degree,lettres,NaN,No,No,50,suisse,...,1.5,0.5,0.0,cats were mummified in ancient egypt\ncats hav...,2.0,1.0,0.0,0.0,T1 Scary,T2 Cute


# Select Covariates

In [9]:
country_subject_df = pd.read_csv('manual_annotation/arash_major_country.csv')
covariate_df = country_subject_df[['Age', 'Gender', 'Studentship', 'Study level', 'Field of study', 'Occupation', 'Allergy', 'Cats previously', 'Cityness_1', 'Nationality', 'English level']].copy()
study_df.drop(columns=['Age', 'Gender', 'Studentship', 'Study level', 'Field of study', 'Occupation', 'Allergy', 'Cats previously', 'Cityness_1', 'Nationality', 'English level', 'wrong_answers'], inplace=True)
covariate_df.rename(columns={'Cityness_1':'Cityness'}, inplace=True)
covariate_df.drop(columns=['Occupation', 'Gender'], inplace=True)
covariate_df

,Age,Studentship,Study level,Field of study,Allergy,Cats previously,Cityness,Nationality,English level
0,23.0,Yes,Bachelors degree,psychology,No,Yes,82.0,gb,C2
1,22.0,Yes,Bachelors degree,psychology,No,Yes,60.0,ch,B2
2,21.0,Yes,Bachelors degree,psychology,Maybe,Yes,60.0,ch,B2
3,25.0,Yes,Pre-University,philosophy,Maybe,No,86.0,ch,C1
4,22.0,Yes,Bachelors degree,psychology,No,Yes,30.0,ch,B2
...,...,...,...,...,...,...,...,...,...
138,22.0,Yes,Bachelors degree,psychology,Yes,No,50.0,ch,B2
139,20.0,Yes,Bachelors degree,arts,Yes,No,70.0,pt,C1
140,19.0,Yes,Pre-University,psychology,No,No,90.0,pt,B2
141,24.0,Yes,Bachelors degree,arts,No,No,50.0,ch,B2


In [10]:
study_df = pd.concat([covariate_df, study_df], axis=1)
study_df.drop(columns=['condition1_freerecall', 'condition2_freerecall'], inplace=True)
study_df

,Age,Studentship,Study level,Field of study,Allergy,Cats previously,Cityness,Nationality,English level,condition1_recog_pos,...,condition1_freerecall_pos,condition1_freerecall_neg,condition1_freerecall_neut,condition1_freerecall_unrel,condition2_freerecall_pos,condition2_freerecall_neg,condition2_freerecall_neut,condition2_freerecall_unrel,condition1,condition2
0,23.0,Yes,Bachelors degree,psychology,No,Yes,82.0,gb,C2,1.000000,...,2.0,1.0,1.0,0.0,2.5,1.0,0.0,0.0,T1 Dirty,T2 Neut
1,22.0,Yes,Bachelors degree,psychology,No,Yes,60.0,ch,B2,0.916667,...,2.5,1.0,0.5,0.0,1.0,2.0,0.0,0.0,T2 Dirty,T1 Neut
2,21.0,Yes,Bachelors degree,psychology,Maybe,Yes,60.0,ch,B2,0.916667,...,1.5,2.0,0.0,0.0,2.0,1.0,0.0,0.0,T1 Cute,T2 Scary
3,25.0,Yes,Pre-University,philosophy,Maybe,No,86.0,ch,C1,0.833333,...,0.5,0.5,0.5,0.0,2.0,1.0,0.0,0.0,T1 Cute,T2 Dirty
4,22.0,Yes,Bachelors degree,psychology,No,Yes,30.0,ch,B2,0.500000,...,2.0,1.0,0.5,0.0,2.5,1.0,0.0,0.0,T1 Neut,T2 Scary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,22.0,Yes,Bachelors degree,psychology,Yes,No,50.0,ch,B2,0.583333,...,0.0,2.0,0.5,0.5,0.0,0.0,0.0,0.0,T2 Cute,T1 Dirty
139,20.0,Yes,Bachelors degree,arts,Yes,No,70.0,pt,C1,0.666667,...,1.0,1.5,0.5,0.0,2.0,1.0,0.0,0.0,T2 Neut,invalid
140,19.0,Yes,Pre-University,psychology,No,No,90.0,pt,B2,0.916667,...,2.0,1.0,0.5,0.0,0.0,2.5,0.0,0.0,T2 Neut,T1 Scary
141,24.0,Yes,Bachelors degree,arts,No,No,50.0,ch,B2,0.500000,...,1.0,1.5,0.5,0.0,2.0,1.0,0.0,0.0,T1 Scary,T2 Cute


# Unify The Conditions

In [11]:

study_df1 = study_df.copy()
study_df2 = study_df.copy()

study_df1.drop(columns=['condition2_recog_pos', 'condition2_recog_neg', 'condition2_recog_neut', 'condition2_recog_unrel', 'condition2_freerecall_pos', 'condition2_freerecall_neg', 'condition2_freerecall_neut', 'condition2_freerecall_unrel', 'condition2'], inplace=True)
study_df1.rename(columns={'condition1_recog_pos':'recog_pos', 'condition1_recog_neg':'recog_neg', 'condition1_recog_neut':'recog_neut', 'condition1_recog_unrel':'recog_unrel', 'condition1_freerecall_pos':'freerecall_pos', 'condition1_freerecall_neg':'freerecall_neg', 'condition1_freerecall_neut':'freerecall_neut', 'condition1_freerecall_unrel':'freerecall_unrel', 'condition1':'condition'}, inplace=True)
study_df1['condition'] = study_df1['condition'].str.split().str[-1]
study_df1['previous_condition'] = 'Nothing'

study_df2.drop(columns=['condition1_recog_pos', 'condition1_recog_neg', 'condition1_recog_neut', 'condition1_recog_unrel', 'condition1_freerecall_pos', 'condition1_freerecall_neg', 'condition1_freerecall_neut', 'condition1_freerecall_unrel', 'condition1'], inplace=True)
study_df2.rename(columns={'condition2_recog_pos':'recog_pos', 'condition2_recog_neg':'recog_neg', 'condition2_recog_neut':'recog_neut', 'condition2_recog_unrel':'recog_unrel', 'condition2_freerecall_pos':'freerecall_pos', 'condition2_freerecall_neg':'freerecall_neg', 'condition2_freerecall_neut':'freerecall_neut', 'condition2_freerecall_unrel':'freerecall_unrel', 'condition2':'condition'}, inplace=True)
study_df2['condition'] = study_df2['condition'].str.split().str[-1]
study_df2['previous_condition'] = study_df1['condition']

stacked_df = pd.concat([study_df1, study_df2], axis=0, ignore_index=True)
stacked_df['condition'] = stacked_df['condition'].str.split().str[-1]
threshold = stacked_df.shape[1] - 3  # Calculate threshold
stacked_df = stacked_df.dropna(thresh=threshold, ignore_index=True)
stacked_df


,Age,Studentship,Study level,Field of study,Allergy,Cats previously,Cityness,Nationality,English level,recog_pos,recog_neg,recog_neut,recog_unrel,freerecall_pos,freerecall_neg,freerecall_neut,freerecall_unrel,condition,previous_condition
0,23.0,Yes,Bachelors degree,psychology,No,Yes,82.0,gb,C2,1.000000,0.916667,0.666667,0.133333,2.0,1.0,1.0,0.0,Dirty,Nothing
1,22.0,Yes,Bachelors degree,psychology,No,Yes,60.0,ch,B2,0.916667,0.583333,0.666667,0.133333,2.5,1.0,0.5,0.0,Dirty,Nothing
2,21.0,Yes,Bachelors degree,psychology,Maybe,Yes,60.0,ch,B2,0.916667,0.583333,0.333333,0.066667,1.5,2.0,0.0,0.0,Cute,Nothing
3,25.0,Yes,Pre-University,philosophy,Maybe,No,86.0,ch,C1,0.833333,0.833333,0.666667,0.066667,0.5,0.5,0.5,0.0,Cute,Nothing
4,22.0,Yes,Bachelors degree,psychology,No,Yes,30.0,ch,B2,0.500000,0.500000,1.000000,0.066667,2.0,1.0,0.5,0.0,Neut,Nothing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,20.0,Yes,Bachelors degree,arts,No,Yes,44.0,ch,B2,0.500000,0.583333,0.333333,0.266667,2.5,0.0,0.0,0.5,Dirty,Neut
280,22.0,Yes,Bachelors degree,psychology,Yes,No,50.0,ch,B2,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,Dirty,Cute
281,20.0,Yes,Bachelors degree,arts,Yes,No,70.0,pt,C1,1.000000,0.833333,1.000000,0.066667,2.0,1.0,0.0,0.0,invalid,Neut
282,19.0,Yes,Pre-University,psychology,No,No,90.0,pt,B2,0.833333,0.833333,0.833333,0.000000,0.0,2.5,0.0,0.0,Scary,Neut


In [12]:
stacked_df.to_csv("final_data_main.csv")